In [1]:
# Install ultralytics YOLOv8
!pip install ultralytics --quiet

# Import libraries
import cv2
from ultralytics import YOLO
from google.colab import files
import time


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 33.7 MB/s eta 0:00:00
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [2]:
# Upload your video
uploaded = files.upload()

# Get uploaded video file name
video_file = list(uploaded.keys())[0]
print("Uploaded video:", video_file)


Saving 12821026_3840_2160_60fps.mp4 to 12821026_3840_2160_60fps.mp4
Uploaded video: 12821026_3840_2160_60fps.mp4


In [3]:
# Load YOLOv8 model (nano version for faster processing)
model = YOLO("yolov8n.pt")  # you can use yolov8s.pt for slightly better accuracy


In [4]:
# Open the uploaded video
cap = cv2.VideoCapture(video_file)

# Output video settings
output_file = f"output_traffic_{int(time.time())}.mp4"
fourcc = cv2.VideoWriter_fourcc(*"mp4v")
out = cv2.VideoWriter(
    output_file, fourcc, int(cap.get(cv2.CAP_PROP_FPS)),
    (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)),
     int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)))
)

print("Output video will be saved as:", output_file)


Output video will be saved as: output_traffic_1768637487.mp4


In [5]:
# Line for counting vehicles
LINE_Y = 350

# Sets for keeping track of counted vehicle IDs
counted_ids = set()
vehicle_count = 0

# Function to get the center of bounding box
def get_center(x1, y1, x2, y2):
    cx = int((x1 + x2)/2)
    cy = int((y1 + y2)/2)
    return cx, cy


In [6]:
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Resize frame for faster YOLO processing
    frame_resized = cv2.resize(frame, (640, 360))

    # Run YOLOv8 vehicle tracking
    results = model.track(
        frame_resized,
        persist=True,        # track IDs across frames
        conf=0.4,            # confidence threshold
        classes=[2,3,5,7],  # car, bike, bus, truck
        verbose=False        # hide speed/inference logs
    )

    # Map coordinates to original frame size
    h_ratio = frame.shape[0] / 360
    w_ratio = frame.shape[1] / 640

    if results[0].boxes.id is not None:
        boxes = results[0].boxes.xyxy.cpu().numpy()
        ids   = results[0].boxes.id.cpu().numpy()
        cls   = results[0].boxes.cls.cpu().numpy()

        for box, track_id, cl in zip(boxes, ids, cls):
            x1, y1, x2, y2 = box
            x1 = int(x1 * w_ratio)
            y1 = int(y1 * h_ratio)
            x2 = int(x2 * w_ratio)
            y2 = int(y2 * h_ratio)

            cx, cy = get_center(x1, y1, x2, y2)

            # Draw bounding box and label
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0,255,0), 2)
            label = f"ID:{int(track_id)} {model.names[int(cl)]}"
            cv2.putText(frame, label, (x1, y1-10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255,255,255), 2)

            # Vehicle counting
            if cy > LINE_Y and track_id not in counted_ids:
                counted_ids.add(track_id)
                vehicle_count += 1

            # Draw center point
            cv2.circle(frame, (cx, cy), 4, (0,0,255), -1)

    # Draw counting line
    cv2.line(frame, (0, LINE_Y), (frame.shape[1], LINE_Y), (255,0,0), 3)

    # Show total vehicle count on frame
    cv2.putText(frame, f"Vehicle Count: {vehicle_count}", (20,50),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,255), 2)

    # Write processed frame to output video
    out.write(frame)


requirements: Ultralytics requirement ['lap>=0.5.12'] not found, attempting AutoUpdate...
Using Python 3.12.12 environment at: /usr
Resolved 2 packages in 171ms
Prepared 1 package in 56ms
Installed 1 package in 1ms
 + lap==0.5.12

requirements: AutoUpdate success ✅ 0.7s
WARNING ⚠️ requirements: Restart runtime or rerun command for updates to take effect



In [7]:
cap.release()
out.release()

# Download the output video
files.download(output_file)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>